## TO DO
* Reproduce age prediction
* Reduce data (sampling resolution + precision)
* Reproduce again

### reproducing CNP_func_preprocess_corr_matrix
https://github.com/danielmlow/brain_age/blob/master/CNP_func_preprocess_corr_matrix.ipynb

In [5]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os
from glob import glob

In [2]:
toy = False
run_all = True
path = '/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/'
output_dir = '/data1/users/jvogel/nilearn_tutorial/output/'
input_dir = '/data1/users/jvogel/nilearn_tutorial/'

n_jobs = 2 #amount of cores #orignally 20
cv=4
description = 'CNP_func_gridsearch_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
log_file

'CNP_func_gridsearch_age_18-09-19-17-45-53'

In [11]:
# Load subject ids
scandirs = sorted(glob(os.path.join(path,'*/')))
subids = [x.split('/')[-2] for x in scandirs]
subids

['sub-10304',
 'sub-10316',
 'sub-10321',
 'sub-10325',
 'sub-10329',
 'sub-10339',
 'sub-10340',
 'sub-10345',
 'sub-10347',
 'sub-10356',
 'sub-10361',
 'sub-10365',
 'sub-10376',
 'sub-10377',
 'sub-10388',
 'sub-10624',
 'sub-10629',
 'sub-10631',
 'sub-10638',
 'sub-10668',
 'sub-10672',
 'sub-10674',
 'sub-10678',
 'sub-10680',
 'sub-10686',
 'sub-10692',
 'sub-10696',
 'sub-10697',
 'sub-10704',
 'sub-10707',
 'sub-10708',
 'sub-10719',
 'sub-10724',
 'sub-10746',
 'sub-10762',
 'sub-10779',
 'sub-10785',
 'sub-10788',
 'sub-10844',
 'sub-10855',
 'sub-10871',
 'sub-10877',
 'sub-10882',
 'sub-10891',
 'sub-10893',
 'sub-10912',
 'sub-10934',
 'sub-10940',
 'sub-10948',
 'sub-10949',
 'sub-10958',
 'sub-10963',
 'sub-10968',
 'sub-10975',
 'sub-10977',
 'sub-10987',
 'sub-10998',
 'sub-11019',
 'sub-11030',
 'sub-11044',
 'sub-11050',
 'sub-11052',
 'sub-11059',
 'sub-11061',
 'sub-11062',
 'sub-11066',
 'sub-11067',
 'sub-11068',
 'sub-11077',
 'sub-11082',
 'sub-11088',
 'sub-

In [16]:
len(subids)

226

Okay that's weird.. seems we have about 100 more subjects than they do...
Later on they do a train-test split -- could they have already left out 100 subjects before this notebook? 
Need to check this.

In [13]:
df = pd.read_csv('/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.4/participants.tsv',
                    sep = '\t')
df.head()

,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,scap,stopsignal,T1w,taskswitch,ScannerSerialNumber
0,sub-10159,CONTROL,30,F,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0
1,sub-10171,CONTROL,24,M,1,1,1,n/a,n/a,1,1,1,1,1,35343.0
2,sub-10189,CONTROL,49,M,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0
3,sub-10193,CONTROL,40,M,1,n/a,1,n/a,n/a,n/a,n/a,n/a,1,n/a,35343.0
4,sub-10206,CONTROL,21,M,1,n/a,1,n/a,n/a,1,1,1,1,1,35343.0


In [15]:
y_age = df.loc[df.participant_id.isin(subids),'age'].values
y_age

array([23, 29, 34, 29, 22, 45, 32, 25, 25, 43, 42, 24, 42, 49, 50, 28, 31,
       21, 25, 25, 29, 42, 24, 22, 26, 28, 25, 39, 32, 28, 25, 23, 22, 23,
       41, 38, 36, 35, 26, 47, 41, 23, 23, 21, 25, 44, 37, 25, 22, 27, 30,
       45, 24, 32, 47, 26, 49, 47, 40, 40, 48, 26, 22, 45, 49, 37, 23, 39,
       22, 25, 21, 24, 43, 39, 21, 27, 27, 21, 29, 40, 28, 38, 24, 28, 32,
       25, 37, 40, 44, 47, 48, 41, 30, 22, 23, 49, 25, 41, 33, 49, 48, 30,
       49, 28, 43, 43, 29, 32, 26, 40, 44, 43, 37, 23, 37, 45, 47, 23, 23,
       49, 38, 39, 41, 48, 31, 33, 25, 23, 47, 33, 42, 29, 29, 32, 40, 25,
       48, 50, 22, 36, 40, 31, 36, 44, 22, 26, 46, 22, 45, 47, 36, 23, 31,
       45, 23, 26, 34, 40, 26, 38, 40, 22, 36, 42, 21, 46, 42, 35, 23, 25,
       37, 43, 46, 37, 25, 50, 34, 29, 48, 47, 33, 29, 32, 41, 29, 50, 44,
       26, 32, 48, 50, 24, 50, 33, 21, 37, 24, 40, 21, 37, 30, 32, 23, 25,
       21, 27, 22, 25, 29, 21, 31, 24, 23, 40, 26, 21, 27, 26, 32, 48, 23,
       21, 48, 50, 46, 48

In [23]:
filenames = sorted(glob(os.path.join(path,'sub-*/func/*MNI*preproc.nii.gz')))
confounds = sorted(glob(os.path.join(path,'sub-*/func/*confounds.tsv')))
print(len(filenames), len(confounds))

223 223


In [24]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')

# We show here networks of 64, 197, 444
networks_64 = parcellations['scale064']
networks_197 = parcellations['scale197']
networks_444 = parcellations['scale444']


Dataset created in /home/users/jvogel/nilearn_data/basc_multiscale_2015



Downloaded 193985 of 193985 bytes (100.0%,    0.0s remaining) ...done. (2 seconds, 0 min)
Extracting data from /home/users/jvogel/nilearn_data/basc_multiscale_2015/3cbcf0eeb3f666f55070aba1db9a758f/1861819..... done.


In [ ]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec

def func_2_corr_matrix(parcellation, name):
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,
                               memory='nilearn_cache', verbose=0)
    #n_regions = 197
    
    correlations = []
    upper_triang_corrs = []
    # Initializing ConnectivityMeasure object with kind='correlation'
    connectome_measure = ConnectivityMeasure(kind='correlation')
    for filename in filenames:
        timeseries_each_subject = masker.fit_transform(filename)
        # call fit_transform from ConnectivityMeasure object
        correlation = connectome_measure.fit_transform([timeseries_each_subject])
        # saving each subject correlation to correlations
        correlations.append(correlation)
        # get the upper triangular correlation matrix of correlation
        upper_triang_corr = sym_matrix_to_vec(correlation, discard_diagonal=True)
        # saving each subject upper_triang_corr to upper_triang_corrs
        upper_triang_corrs.append(upper_triang_corr) 
        print(filename,'finished')
    #   Split train test sets and save
    X_train = upper_triang_corrs[:84]
    X_test = upper_triang_corrs[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = subj[:84]
    random_subj_test = subj[84:]
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)
    return correlations, upper_triang_corrs

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, '197')
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, '444')

/home/users/jvogel/anaconda2/envs/py3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:462: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10304/func/sub-10304_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10316/func/sub-10316_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10321/func/sub-10321_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10325/func/sub-10325_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10329/func/sub-10329_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10339/func/sub-10339_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10963/func/sub-10963_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10968/func/sub-10968_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10975/func/sub-10975_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10977/func/sub-10977_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10987/func/sub-10987_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-10998/func/sub-10998_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50029/func/sub-50029_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50032/func/sub-50032_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50033/func/sub-50033_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50034/func/sub-50034_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50035/func/sub-50035_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-50036/func/sub-50036_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo

/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60033/func/sub-60033_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60036/func/sub-60036_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60037/func/sub-60037_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60038/func/sub-60038_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60042/func/sub-60042_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvogel/nilearn_tutorial/ds000030_R1.0.5/derivatives/fmriprep/sub-60043/func/sub-60043_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz finished
/data1/users/jvo